In [1]:
!pip install pyomo
!pip install cplex -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 14.6 MB/s eta 0:00:00


In [2]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [15]:
model = pyo.ConcreteModel()

model.i = pyo.Set(initialize = ['shirts','shorts','pants'])
model.P = pyo.Param(model.i, initialize = {'shirts':12,'shorts':8,'pants':15})
P = model.P
model.Co = pyo.Param(model.i, initialize = {'shirts':6,'shorts':4,'pants':8})
Co = model.Co
model.F = pyo.Param(model.i, initialize = {'shirts':200,'shorts':150,'pants':100})
F = model.F
model.L = pyo.Param(model.i, initialize = {'shirts':3,'shorts':2,'pants':6})
L = model.L
model.C = pyo.Param(model.i, initialize = {'shirts':4,'shorts':3,'pants':4})
C = model.C
model.M = pyo.Param(model.i, initialize = {'shirts':40,'shorts':53,'pants':25})
M = model.M

model.x = pyo.Var(model.i, domain = pyo.Integers)
model.y = pyo.Var(model.i, domain = pyo.Binary)
x = model.x
y = model.y

def objective_rule(model,i):
  return sum(P[i]*x[i] for i in model.i) - sum(Co[i]*x[i] for i in model.i) - sum(F[i]*y[i] for i in model.i)
model.objective = pyo.Objective(rule = objective_rule, sense = pyo.maximize)

def constr1(model,i):
  return sum(L[i]*x[i] for i in model.i) <= 150
model.constraint1 = pyo.Constraint(rule = constr1)

def constr2(model,i):
  return sum(C[i]*x[i] for i in model.i) <= 160
model.constraint2 = pyo.Constraint(rule = constr2)

def constr3(model,i):
  return x[i] <= M[i] * y[i]
model.constraint3 = pyo.Constraint(model.i,rule = constr3)

solver = SolverFactory('cplex_direct')
results = solver.solve(model)

print(results)
print('Objective function =',model.objective())
for i in model.i:
  print('Number of',i,'produces each week =',x[i]())




Problem: 
- Name: 
  Lower bound: 75.00000000000031
  Upper bound: 75.00000000000031
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 6
  Number of binary variables: 3
  Number of integer variables: 3
  Number of continuous variables: 0
  Number of nonzeros: None
  Sense: -1
Solver: 
- Name: CPLEX 22.1.1.0
  Status: ok
  Wallclock time: 0.005083560943603516
  Termination condition: optimal
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Objective function = 75.00000000000034
Number of shirts produces each week = 0.0
Number of shorts produces each week = 1.1368683772161603e-13
Number of pants produces each week = 24.99999999999996
